In [21]:
import random
import string
import time

# 1. Przyjmij następujący zbiór danych wejściowych:

In [22]:
data1 = "bbb$"
data2 = "aabbabd"
data3 = "ababcd"
data4 = "abaababaabaabaabab$"
data5 = ''.join(random.choices(string.ascii_lowercase, k=100))
data6 = None

with open('1997_714_head.txt', 'r') as file:
    data6 = file.read()

# 2. Upewnij się, że każdy łańcuch na końcu posiada unikalny znak (marker), a jeśli go nie ma, to dodaj ten znak.

In [23]:
def marker_check(text, marker='$'):
    lines = text.splitlines()
    processed = []

    for line in lines:
        if not line.endswith(marker):
            line += marker
        processed.append(line)

    return '\n'.join(processed)


test_data = "abc"
print("Przed: " + test_data)
processed_data = marker_check(test_data)
print("Po: " + processed_data)
print("-------------------")

test_data = "abcd$"
print("Przed: " + test_data)
processed_data = marker_check(test_data)
print("Po: " + processed_data)
print("-------------------")


Przed: abc
Po: abc$
-------------------
Przed: abcd$
Po: abcd$
-------------------


# 3. Zaimplementuj algorytm konstruujący strukturę trie, która przechowuje wszystkie sufiksy łańcucha danego na wejściu.

Zwykłe Trie (bez sufix linków)

In [24]:
class TrieNode1:
    def __init__(self, parent=None):
        self.parent = parent
        self.children = {}

class Trie1:
    def __init__(self, text):
        self.root = TrieNode1()
        self.build(text)

    def build(self, text):
        n = len(text)
        for i in range(n):
            suffix = text[i:]
            node = self.root
            for char in suffix:
                if char not in node.children:
                    new = TrieNode1(node)
                    node.children[char] = new
                node = node.children[char]

    def search(self, pattern):
        node = self.root
        for char in pattern:
            if char not in node.children:
                return False
            node = node.children[char]
        return True

Trie z suffix linkami:

In [25]:
class TrieNode2: 
    def __init__(self, parent=None):
        self.parent = parent
        self.children = {}
        self.link = None

class Trie2: 
    def __init__(self, text):
        self.root = TrieNode2(None)
        self.build(text)

    def build(self, text):
        low = self.root 
        
        for char in text:

            node, low, prev = low, None, None
            
            while node is not None and char not in node.children:

                new_child = TrieNode2(node)
                node.children[char] = new_child
                

                if node == self.root: 
                    new_child.link = self.root 
                if not low: 
                    low = new_child
                if prev: 
                    prev.link = new_child
                    
                prev = new_child
                node = node.link
                
            else:

                if node is not None:
                    prev.link = node.children[char]


    def search(self, pattern):
        node = self.root
        for char in pattern:
            if char not in node.children:
                return False
            node = node.children[char]
        return True

# 4. Zaimplementuj algorytm konstruujący drzewo sufiksów.

In [26]:
#TODO

# 5. Upewnij się, że powstałe struktury danych są poprawne. Możesz np. sprawdzić, czy struktura zawiera jakiś ciąg znaków i porównać wyniki z algorytmem wyszukiwania wzorców.

Algorytm wyszukiwania wzorców (Algorytm Knutha-Morrisa-Pratta) został wzięty z laboratorium pierwszego.

In [27]:
def prefix_function(pattern):
    m = len(pattern)
    pi = [0] * m
    k = 0

    for q in range(1, m):
        while k > 0 and pattern[k] != pattern[q]:
            k = pi[k-1]
        if pattern[k] == pattern[q]:
            k += 1
        pi[q] = k

    return pi


def KMP(text, pattern):
    pi = prefix_function(pattern)
    q = 0
    result = []
    
    for i in range(0, len(text)):
        while q > 0 and pattern[q] != text[i]:
            q = pi[q-1]
        if pattern[q] == text[i]:
            q += 1
        if q == len(pattern):
            q = pi[q-1]
            result.append(i+1-q)
    return result

Funkcja testująca:

In [28]:
def test_(text, pattern):
    trie1 = Trie1(text)
    trie_occurrence1 = trie1.search(pattern)
    
    trie2 = Trie2(text)
    trie_occurrence2 = trie2.search(pattern)
    
    occ = KMP(text, pattern)
    kmp_occurrence = len(occ) > 0
    if kmp_occurrence and trie_occurrence2:
        print("Znaleziono wzorzec.")
    else:
        print("Nie znaleziono wzorca.")

        
test_("bananasbananas", "ananas")
test_("aaaaaaaaaa", "a")
test_("bananasbananas", "banana")
test_("asd", "sd")

Znaleziono wzorzec.
Znaleziono wzorzec.
Znaleziono wzorzec.
Znaleziono wzorzec.


# 6. Porównaj szybkość działania algorytmów konstruujących struktury danych dla danych z p. 1 w następujących wariantach:
<ul>
<li>Trie - w wariancie, w którym kolejne sufiksy dodawane są przez przeszukiwanie głowy od korzenia drzewa (1p.),</li>
<li>Trie - w wariancie, w którym kolejne sufiksy dodawane są poprzez dodanie kolejnej litery tekstu (1p.),</li>
    <li>Drzewo sufiksów - algorytm Ukkonena (3p).</li>
    </ul>

In [29]:
# ad1: Trie1 - w wariancie, w którym kolejne sufiksy dodawane
# są przez przeszukiwanie głowy od korzenia drzewa

# ad2: Trie2 - w wariancie, w którym kolejne sufiksy dodawane 
# są poprzez dodanie kolejnej litery tekstu

def measure_time(data):
    start_time = time.time()
    trie1 = Trie1(data)
    time_trie1 = time.time() - start_time
    
    start_time = time.time()
    trie2 = Trie2(data)
    time_trie2 = time.time() - start_time
    
    print(f"Czas konstrukcji Trie z zadania 1: {time_trie1} s")
    print(f"Czas konstrukcji Trie z zadania 2: {time_trie2} s")
    
    if time_trie1 > time_trie2:
        print("Szybciej utworzono Trie2")
    else:
        print("Szybciej utworzono Trie1")
    
print("Tests for data1")
print("----------------------")
measure_time(data1)
print("")
print("Tests for data2")
print("----------------------")
measure_time(data2)
print("")
print("Tests for data3")
print("----------------------")
measure_time(data3)
print("")
print("Tests for data4")
print("----------------------")
measure_time(data4)
print("")
print("Tests for data5")
print("----------------------")
measure_time(data5)
print("")
print("Tests for data6")
print("----------------------")
measure_time(data6)

Tests for data1
----------------------
Czas konstrukcji Trie z zadania 1: 1.5020370483398438e-05 s
Czas konstrukcji Trie z zadania 2: 4.8160552978515625e-05 s
Szybciej utworzono Trie1

Tests for data2
----------------------
Czas konstrukcji Trie z zadania 1: 0.000186920166015625 s
Czas konstrukcji Trie z zadania 2: 2.288818359375e-05 s
Szybciej utworzono Trie2

Tests for data3
----------------------
Czas konstrukcji Trie z zadania 1: 2.384185791015625e-05 s
Czas konstrukcji Trie z zadania 2: 2.002716064453125e-05 s
Szybciej utworzono Trie2

Tests for data4
----------------------
Czas konstrukcji Trie z zadania 1: 0.0001232624053955078 s
Czas konstrukcji Trie z zadania 2: 0.00018477439880371094 s
Szybciej utworzono Trie1

Tests for data5
----------------------
Czas konstrukcji Trie z zadania 1: 0.005864381790161133 s
Czas konstrukcji Trie z zadania 2: 0.006970882415771484 s
Szybciej utworzono Trie1

Tests for data6
----------------------
Czas konstrukcji Trie z zadania 1: 18.40722703933